In [4]:
import pandas as pd

# from tqdm import tqdm
from tqdm.auto import tqdm
from tqdm import tqdm

tqdm.pandas()

/root/nielsen/churn/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
abonnementsprodukter__historikk_df = pd.read_csv(
    "received_csvs/abonnementsprodukter__historikk.csv"
)


df_part1 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part1.csv")
df_part2 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part2.csv")
df_part3 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part3.csv")
df_part4 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part4.csv")
df_part5 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part5.csv")
df_part6 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part6.csv")
eierverifiseringer__historikk_df = pd.concat(
    [df_part1, df_part2, df_part3, df_part4, df_part5, df_part6], ignore_index=True
)

produktuttak_df = pd.read_csv("received_csvs/produktuttak.csv")
stg_dynamics__contacts_df = pd.read_csv("received_csvs/stg_dynamics__contacts.csv")
nps_svar_df = pd.read_csv("received_csvs/nps_svar.csv").drop(columns="contact_rk.1")

/tmp/ipykernel_1221309/2077797385.py:8: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_part3 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part3.csv")
/tmp/ipykernel_1221309/2077797385.py:10: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_part5 = pd.read_csv("received_csvs/eierverifiseringer__historikk_part5.csv")


In [6]:
abonnementsprodukter__historikk_df.rename(
    columns={"subscriptionitem__contact_rk": "contact_rk"}, inplace=True
)

In [7]:
len(eierverifiseringer__historikk_df)

10444260

### Only store rows with valid membership

In [8]:
eierverifiseringer__historikk_df = eierverifiseringer__historikk_df[
    eierverifiseringer__historikk_df["contact_rk"].isin(
        abonnementsprodukter__historikk_df["contact_rk"]
    )
]
produktuttak_df = produktuttak_df[
    produktuttak_df["contact_rk"].isin(abonnementsprodukter__historikk_df["contact_rk"])
]
stg_dynamics__contacts_df = stg_dynamics__contacts_df[
    stg_dynamics__contacts_df["contact_rk"].isin(
        abonnementsprodukter__historikk_df["contact_rk"]
    )
]
nps_svar_df = nps_svar_df[
    nps_svar_df["contact_rk"].isin(abonnementsprodukter__historikk_df["contact_rk"])
]

In [9]:
len(eierverifiseringer__historikk_df)

7684725

# Cleanup

In [11]:
# Count NaNs in specific columns
nan_counts = (
    abonnementsprodukter__historikk_df[["innmeldingsdato", "startdato", "sluttdato"]]
    .isna()
    .sum()
)

print("Number of NaNs:")
print(nan_counts)

Number of NaNs:
innmeldingsdato    102811
startdato               0
sluttdato          524027
dtype: int64


In [12]:
# Find duplicated contact_rk values
duplicates = stg_dynamics__contacts_df["contact_rk"].value_counts().loc[lambda x: x > 1]

# Display how many contact_rk are duplicated
print(f"Number of non-unique contact_rk: {duplicates.shape[0]}")

# If you want to see the list:
print(duplicates)

Number of non-unique contact_rk: 0
Series([], Name: count, dtype: int64)


In [13]:
abonnementsprodukter__historikk_df = abonnementsprodukter__historikk_df.dropna(
    subset=["sluttdato"]
)
abonnementsprodukter__historikk_df = abonnementsprodukter__historikk_df.drop(
    columns=["innmeldingsdato", "avtale_antall_år"]
)
nan_count = stg_dynamics__contacts_df[["age"]].isna().sum()
stg_dynamics__contacts_df = stg_dynamics__contacts_df.dropna(subset=["age"])

print("Rows with age NaN dropped:\n", nan_count)

Rows with age NaN dropped:
 age    58195
dtype: int64


In [15]:
eierverifiseringer__historikk_df.columns

Index(['registreringsnummer', 'status_kode_id', 'contact_rk',
       'ownerverification__vehicle_rk', 'vehicle_rk', 'merke', 'modell',
       'modelltype', 'kjøretøytype', 'drivstoff', 'girkasse', 'status',
       'antall_aksler', 'antall_passasjerer', 'slagvolum_ccm', 'slagvolum_l',
       'motoreffekt_hk', 'årsmodell', 'antall_år_registrert_på_eier',
       'vraket_dato', 'er_importert', 'forrige_eu_kontroll_dato',
       'neste_eu_kontroll_dato', 'registert_første_gang_dato'],
      dtype='object')

# Data processing

In [ ]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

df = abonnementsprodukter__historikk_df.copy()
df["startdato"] = pd.to_datetime(df["startdato"], errors="coerce")
df["sluttdato"] = pd.to_datetime(df["sluttdato"], errors="coerce")
df = df.dropna(subset=["startdato", "sluttdato"])


# make sure each contact is sorted
df = df.sort_values(["contact_rk", "startdato"])


# define the two product‐keys
young_code = "<redacted>"
reg_code = "<redacted>"

# rebuild one “row per contact” (merging the 2 if they qualify)
records = []
for contact, grp in tqdm(
    df.groupby("contact_rk"), desc="Processing contacts", mininterval=0.1
):
    has_y = young_code in grp["subscriptionitem__dynamics_product_rk"].values
    has_r = reg_code in grp["subscriptionitem__dynamics_product_rk"].values

    # merge case
    if has_y and has_r:
        y = (
            grp[grp["subscriptionitem__dynamics_product_rk"] == young_code]
            .sort_values("startdato")
            .iloc[-1]
        )
        r = (
            grp[grp["subscriptionitem__dynamics_product_rk"] == reg_code]
            .sort_values("startdato")
            .iloc[0]
        )

        if (r["startdato"] - y["sluttdato"]).days <= 365:
            merged = r.copy()
            merged["startdato"] = y["startdato"]
            merged["sluttdato"] = r["sluttdato"]
            merged["regular_start"] = r["startdato"]
            merged["young_to_regular"] = True  # <— flag it
            records.append(merged)
            continue

    # all other cases
    for _, row in grp.iterrows():
        r2 = row.copy()
        r2["regular_start"] = (
            r2["startdato"]
            if r2["subscriptionitem__dynamics_product_rk"] == reg_code
            else pd.NaT
        )
        r2["young_to_regular"] = False  # <— no merge
        records.append(r2)

df2 = pd.DataFrame(records)

# feature‐engineering
# precompute who ever had each type
contact_codes = df.groupby("contact_rk")["subscriptionitem__dynamics_product_rk"].agg(
    set
)

had_young = contact_codes.map(lambda s: young_code in s)
had_regular = contact_codes.map(lambda s: reg_code in s)

df2["had_young_member_flag"] = df2["contact_rk"].map(
    lambda x: True if had_young[x] else False
)

df2["only_young"] = df2["contact_rk"].map(lambda x: had_young[x] and not had_regular[x])

Processing contacts: 100%|██████████| 431462/431462 [04:40<00:00, 1538.30it/s]


In [ ]:
# Compute “prev_end” and gap
df2["prev_end"] = df2.groupby("contact_rk")["sluttdato"].shift()
df2["gap"] = (df2["startdato"] - df2["prev_end"]).dt.days.fillna(-1).astype(int)

# Flag new member-instance when gap > 365 or first record
df2["new_instance"] = df2["gap"] > 365

# Instance index per contact
df2["instance_idx"] = df2.groupby("contact_rk")["new_instance"].cumsum()

# Aggregate per (contact_rk, instance_idx)
agg = {
    "startdato": "min",
    "sluttdato": "max",
    "gap": "first",
}
df_agg = df2.groupby(["contact_rk", "instance_idx"]).agg(agg).reset_index()

# Churn metrics
df_agg["churn_count"] = df_agg["instance_idx"]
df_agg["last_churn_gap"] = df_agg["gap"].where(df_agg["instance_idx"] > 1, -1)

# Create a new contact_rk identifier per instance
df_agg["contact_rk_new"] = (
    df_agg["contact_rk"].astype(str) + "_" + df_agg["instance_idx"].astype(str)
)

# Merge back instance-related fields to original df
df2 = df2.merge(
    df_agg[
        [
            "contact_rk",
            "instance_idx",
            "churn_count",
            "last_churn_gap",
            "contact_rk_new",
        ]
    ],
    on=["contact_rk", "instance_idx"],
    how="left",
)

In [ ]:
# Calculate adjusted_date and target
def calculate_adjusted_date_and_target(row):
    start = row["startdato"]
    end = row["sluttdato"]

    # Duration in days
    duration = (end - start).days

    # Define year thresholds
    one_year_3_weeks = 365 + 21
    two_years_3_weeks = 2 * 365 + 21
    three_years_3_weeks = 3 * 365 + 21

    # Valid target options
    valid_choices = [0]
    if duration >= one_year_3_weeks:
        valid_choices.append(1)
    if duration >= two_years_3_weeks:
        valid_choices.append(2)

    years_to_subtract = np.random.choice(valid_choices)

    # Anchor to Feb 2nd of the end year, then subtract years
    adjusted_base = pd.Timestamp(year=end.year, month=2, day=2)
    adjusted_date = adjusted_base - relativedelta(years=years_to_subtract)

    return pd.Series(
        [adjusted_date, years_to_subtract], index=["adjusted_date", "target"]
    )


df2[["adjusted_date", "target"]] = df2.progress_apply(
    calculate_adjusted_date_and_target, axis=1
)

# Handle adjusted_date < startdato
# Drop these rows
df2 = df2[df2["adjusted_date"] >= df2["startdato"]]

100%|██████████| 534585/534585 [01:44<00:00, 5129.40it/s]


In [ ]:
# Calculate membership duration
def calculate_membership_duration(row):
    start = row["startdato"]
    adjusted = row["adjusted_date"]
    delta = relativedelta(adjusted, start)
    full_years = delta.years
    total_days = (adjusted - start).days
    return pd.Series(
        [full_years, total_days], index=["membership_years", "membership_days"]
    )


# now ONLY compute days_since_regular on the merged rows:
df2["days_since_regular"] = np.where(
    df2["young_to_regular"],
    (df2["adjusted_date"] - df2["regular_start"]).dt.days,
    -1,
)

df2[["membership_years", "membership_days"]] = df2.progress_apply(
    calculate_membership_duration, axis=1
)

100%|██████████| 449483/449483 [01:13<00:00, 6132.56it/s]


In [26]:
df2 = df2.drop(
    columns=[
        "prev_end",
        "regular_start",
        "subscriptionitem__dynamics_product_rk",
        "instance_idx",
        "new_instance",
    ]
)

# Vehicle

In [ ]:
import json

price_dict = {}

with open("car_mapping_final.json", "r") as f:
    price_dict = json.load(f)

In [ ]:
df_car = eierverifiseringer__historikk_df.copy()

# Compute average price for each car entry
average_prices = {k: np.mean(v) for k, v in price_dict.items()}


def simplify_modell(modell):
    if pd.isnull(modell):
        return ""
    return modell.strip().split()[0]


df_car["simplified_modell"] = df_car["modell"].apply(simplify_modell)


# Adjust the make_price_key function to use the simplified model
def make_price_key(row):
    try:
        return f"{row['merke'].strip()} {row['simplified_modell']} {row['drivstoff'].strip()} {int(row['årsmodell'])}"
    except:
        return None


df_car["price_key"] = df_car.progress_apply(make_price_key, axis=1)


# Define depreciation function
def estimate_original_price_piecewise(row):
    key = row["price_key"]
    years = row["antall_år_registrert_på_eier"]

    if key not in average_prices or pd.isnull(years):
        return -1

    current_price = average_prices[key]

    if years < 1:
        return -1  # not enough info

    elif years == 1:
        return round(current_price / 0.80, 2)
    elif years == 2:
        return round(current_price / 0.70, 2)
    elif years == 3:
        return round(current_price / 0.60, 2)
    elif years == 4:
        return round(current_price / 0.54, 2)
    elif years == 5:
        return round(current_price / 0.486, 2)
    elif years >= 6:
        decay_factor = 0.486 * (0.90 ** (years - 5))
        return round(current_price / decay_factor, 2)

    return -1


# Apply estimation
df_car["estimated_first_owner_price"] = df_car.progress_apply(
    estimate_original_price_piecewise, axis=1
)

/root/nielsen/churn/.venv/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/root/nielsen/churn/.venv/lib/python3.11/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 7684725/7684725 [01:30<00:00, 84902.12it/s]


In [32]:
df_car = df_car.drop(
    columns=[
        "modelltype",
        "kjøretøytype",
        "antall_aksler",
        "slagvolum_ccm",
        "slagvolum_l",
        "price_key",
        "simplified_modell",
    ]
)

In [ ]:
import pandas as pd

# Ensure the dates are datetimes
df_car["neste_eu_kontroll_dato"] = pd.to_datetime(
    df_car["neste_eu_kontroll_dato"], errors="coerce"
)

# Bring membership_years onto the car table
df_car2 = df_car.merge(
    df2[["contact_rk", "target"]],
    on="contact_rk",
    how="right",  # keep contacts even if they have zero cars
    indicator=True,  # so we know who had no cars at all
)

# Filter out ineligible cars
df_car2 = df_car2[df_car2["antall_år_registrert_på_eier"] <= df_car2["target"]]

# Calculate days until next EU-kontroll
df_car2["days_to_eu"] = (
    df_car2["neste_eu_kontroll_dato"].sub(pd.Timestamp.now()).dt.days
)

# Compute per-contact summaries
#  5a) Count of valid cars
count = df_car2.groupby("contact_rk").size().rename("num_cars")

#  Identify the row-index of the car with the max årsmodell
idx_max = df_car2.groupby("contact_rk")["årsmodell"].idxmax()

#  Pull out that “latest” car’s details
latest = (
    df_car2.loc[idx_max]
    .set_index("contact_rk", drop=False)[
        [
            "merke",
            "modell",
            "drivstoff",
            "girkasse",
            "antall_passasjerer",
            "motoreffekt_hk",
            "årsmodell",
            "antall_år_registrert_på_eier",
            "days_to_eu",
            "estimated_first_owner_price",
        ]
    ]
    .rename(
        columns={
            "merke": "latest_merke",
            "modell": "latest_modell",
            "drivstoff": "latest_drivstoff",
            "girkasse": "latest_girkasse",
            "antall_passasjerer": "latest_antall_passasjerer",
            "motoreffekt_hk": "latest_motoreffekt_hk",
            "årsmodell": "latest_årsmodell",
            "antall_år_registrert_på_eier": "latest_antall_år_registrert_på_eier",
            "days_to_eu": "days_to_neste_eu_kontroll_dato",
            "estimated_first_owner_price": "estimated_first_owner_price",
        }
    )
)

# Merge summaries back into df2
df2_merged = (
    df2.set_index("contact_rk")
    .join(count, how="left")
    .join(latest, how="left")
    .fillna({"num_cars": 0})  # contacts with no eligible cars get zero
    .reset_index()
)

In [35]:
df2_merged["latest_antall_år_registrert_på_eier"] = (
    df2_merged["latest_antall_år_registrert_på_eier"] - df2_merged["target"]
).clip(lower=0)

In [36]:
df2_merged.columns

Index(['contact_rk', 'subscriptionitem_rk', 'er_husstandsmedlem', 'startdato',
       'sluttdato', 'young_to_regular', 'had_young_member_flag', 'only_young',
       'gap', 'churn_count', 'last_churn_gap', 'contact_rk_new',
       'adjusted_date', 'target', 'days_since_regular', 'membership_years',
       'membership_days', 'num_cars', 'latest_merke', 'latest_modell',
       'latest_drivstoff', 'latest_girkasse', 'latest_antall_passasjerer',
       'latest_motoreffekt_hk', 'latest_årsmodell',
       'latest_antall_år_registrert_på_eier', 'days_to_neste_eu_kontroll_dato',
       'estimated_first_owner_price'],
      dtype='object')

# Produktuttak

In [ ]:
# Make sure all your dates and amounts are the right dtype:
produktuttak_df["avsluttet_dato"] = pd.to_datetime(
    produktuttak_df["avsluttet_dato"], errors="coerce"
)
produktuttak_df["beløp"] = pd.to_numeric(produktuttak_df["beløp"], errors="coerce")
df2_merged["adjusted_date"] = pd.to_datetime(
    df2_merged["adjusted_date"], errors="coerce"
)

# Bring adjusted_date onto the produktuttak table so you can filter per-row:
pt = produktuttak_df.merge(
    df2_merged[["contact_rk", "adjusted_date"]], on="contact_rk", how="inner"
)

# Filter out any produktuttak where avsluttet_dato is on/after the adjusted_date:
pt = pt[pt["avsluttet_dato"] < pt["adjusted_date"]]

# Group & aggregate per contact:
spend = (
    pt.groupby("contact_rk")
    .agg(
        total_spent=("beløp", "sum"),
        last_spent_date=("avsluttet_dato", "max"),
    )
    .reset_index()
)

# Compute days since that last spend:
spend["days_since_last_spent"] = (
    pd.Timestamp.now().normalize()  # or use a fixed “today” if you prefer
    - spend["last_spent_date"]
).dt.days

# 6) Merge those two new columns back into df2_merged:
df2_merged = df2_merged.merge(
    spend[["contact_rk", "total_spent", "days_since_last_spent"]],
    on="contact_rk",
    how="left",
)

df2_merged["total_spent"] = df2_merged["total_spent"].fillna(0)
df2_merged["days_since_last_spent"] = df2_merged["days_since_last_spent"].fillna(-1)

# Personal Info

In [ ]:
# Ensure date columns are datetimes
stg_dynamics__contacts_df["birth_date"] = pd.to_datetime(
    stg_dynamics__contacts_df["birth_date"], errors="coerce"
)
df2_merged["adjusted_date"] = pd.to_datetime(
    df2_merged["adjusted_date"], errors="coerce"
)

# Merge in birth_date, gender, sentralitetsindex
df2_merged = df2_merged.merge(
    stg_dynamics__contacts_df[
        ["contact_rk", "birth_date", "gender", "sentralitetsindex"]
    ],
    on="contact_rk",
    how="left",
)

# Calculate age (in whole years) as of adjusted_date
df2_merged["age"] = (
    (df2_merged["adjusted_date"] - df2_merged["birth_date"])
    .dt.days.floordiv(365.25)
    .astype("Int64")  # nullable integer, if you want missing where birth_date is null
)

In [42]:
# Original number of rows
original_count = len(df2_merged)

# Filter the DataFrame
# df2_merged = df2_merged[(df2_merged['age'] >= 18) & (df2_merged['age'] <= 90)]
df2_merged = df2_merged[
    (df2_merged["age"] >= 18) & (df2_merged["age"] <= 90) & df2_merged["age"].notna()
]

# Number of rows after filtering
filtered_count = len(df2_merged)

# Number of rows dropped
dropped_count = original_count - filtered_count

# Display the counts
print(f"Original row count: {original_count}")
print(f"Remaining row count after filtering: {filtered_count}")
print(f"Number of rows dropped: {dropped_count}")

Original row count: 449483
Remaining row count after filtering: 419432
Number of rows dropped: 30051


# NPS Review

In [ ]:
# Ensure dates are datetimes
nps_svar_df["opprettet_dato"] = pd.to_datetime(
    nps_svar_df["opprettet_dato"], errors="coerce"
)
df2_merged["adjusted_date"] = pd.to_datetime(
    df2_merged["adjusted_date"], errors="coerce"
)

# Bring adjusted_date onto the NPS table so we can filter per-row
nps = nps_svar_df.merge(
    df2_merged[["contact_rk", "adjusted_date"]], on="contact_rk", how="inner"
)

# Keep only reviews that happened before adjusted_date
nps = nps[nps["opprettet_dato"] < nps["adjusted_date"]]

# Compute group-level aggregates
agg = (
    nps.groupby("contact_rk")
    .agg(
        nps_mean=("score", "mean"),
        nps_median=("score", "median"),
        nps_count=("score", "size"),
        last_date=("opprettet_dato", "max"),
    )
    .reset_index()
)

# Pull out the score on that last_date (nps_latest)
idx = nps.groupby("contact_rk")["opprettet_dato"].idxmax()
latest = nps.loc[idx, ["contact_rk", "score"]].rename(columns={"score": "nps_latest"})

# Merge mean/median/count/last_date with latest score
nps_summary = agg.merge(latest, on="contact_rk", how="left")

# Compute days since the last review
nps_summary["days_since_nps"] = (
    (
        nps_summary["adjusted_date"]
        if "adjusted_date" in nps_summary
        else nps_summary.merge(
            df2_merged[["contact_rk", "adjusted_date"]], on="contact_rk", how="left"
        )["adjusted_date"]
    )
    .sub(nps_summary["last_date"])
    .dt.days
)

# Bring the summary into df2_merged
df2_merged = df2_merged.merge(
    nps_summary[
        [
            "contact_rk",
            "nps_mean",
            "nps_median",
            "nps_latest",
            "nps_count",
            "days_since_nps",
        ]
    ],
    on="contact_rk",
    how="left",
)

# Fill in contacts with no reviews
df2_merged[["nps_mean", "nps_median", "nps_latest", "days_since_nps"]] = df2_merged[
    ["nps_mean", "nps_median", "nps_latest", "days_since_nps"]
].fillna(-1)

df2_merged["nps_count"] = df2_merged["nps_count"].fillna(0).astype(int)

In [45]:
df2_merged.columns

Index(['contact_rk', 'subscriptionitem_rk', 'er_husstandsmedlem', 'startdato',
       'sluttdato', 'young_to_regular', 'had_young_member_flag', 'only_young',
       'gap', 'churn_count', 'last_churn_gap', 'contact_rk_new',
       'adjusted_date', 'target', 'days_since_regular', 'membership_years',
       'membership_days', 'num_cars', 'latest_merke', 'latest_modell',
       'latest_drivstoff', 'latest_girkasse', 'latest_antall_passasjerer',
       'latest_motoreffekt_hk', 'latest_årsmodell',
       'latest_antall_år_registrert_på_eier', 'days_to_neste_eu_kontroll_dato',
       'estimated_first_owner_price', 'total_spent', 'days_since_last_spent',
       'birth_date', 'gender', 'sentralitetsindex', 'age', 'nps_mean',
       'nps_median', 'nps_latest', 'nps_count', 'days_since_nps'],
      dtype='object')

In [46]:
df_ml = df2_merged.copy()

In [47]:
df_ml = df_ml.drop(
    columns=[
        "contact_rk",
        "subscriptionitem_rk",
        "startdato",
        "sluttdato",
        "adjusted_date",
        "birth_date",
        "contact_rk_new",
    ]
)

In [48]:
df_ml.columns

Index(['er_husstandsmedlem', 'young_to_regular', 'had_young_member_flag',
       'only_young', 'gap', 'churn_count', 'last_churn_gap', 'target',
       'days_since_regular', 'membership_years', 'membership_days', 'num_cars',
       'latest_merke', 'latest_modell', 'latest_drivstoff', 'latest_girkasse',
       'latest_antall_passasjerer', 'latest_motoreffekt_hk',
       'latest_årsmodell', 'latest_antall_år_registrert_på_eier',
       'days_to_neste_eu_kontroll_dato', 'estimated_first_owner_price',
       'total_spent', 'days_since_last_spent', 'gender', 'sentralitetsindex',
       'age', 'nps_mean', 'nps_median', 'nps_latest', 'nps_count',
       'days_since_nps'],
      dtype='object')

In [ ]:
df_ml.to_csv("df_ml.csv")